In [ ]:
import numpy as np
import cmath
from multipledispatch import dispatch
import time
from scipy.optimize import minimize
from scipy.optimize import root

In [ ]:
def create_phase_matrix(n, phase):
    phase_matrix = np.eye(n, dtype = complex)
    for i in range(n-1):
        for j in range(n-1):
            if (i==j):
                phase_matrix[i][j] = cmath.exp(1j*phase[i])
    return phase_matrix


def create_phi_for_each_layer(n):
    phis = [0] * 2 * N
    for _i in range(2 * N):
        phis[_i] += _i * cmath.pi / 2
    del _i
    return phis


def multiply_v_matrices(n, theta, phis):
    _V_matrices_product = np.eye(n, dtype=complex)
    for _i in range(2 * n):  # i is a number of a layer
        _j = _i % 2  # j - even or odd layer
        _V = np.eye(n, dtype=complex)
        while _j < n - 1:  # j became channel number
            _T = np.eye(n, dtype=complex)
            _T[_j][_j] = cmath.exp(1j * phis[_i]) * cmath.cos(theta)
            _T[_j][_j + 1] = cmath.sin(theta)
            _T[_j + 1][_j] = -cmath.exp(1j * phis[_i]) * cmath.sin(theta)
            _T[_j + 1][_j + 1] = cmath.cos(theta)
            _V = np.dot(_V, _T)
            _j += 2
        _V_matrices_product = np.dot(_V, _V_matrices_product)
    del _i, _j, _T, _V
    return _V_matrices_product


def is_unitary(u, precision, n, atol):
    if np.allclose(np.eye(n, dtype=complex), np.around(np.dot(u.H, u), precision), rtol=0, atol=atol):
        return True
    return False

def create_input_vector_of_amplitudes(n):
    _input_vector = []
    for _i in range(n):
        _input_vector.append(_i)  # create input vector of amplitudes
    del _i
    return _input_vector

@dispatch(int)
def create_initial_phases(n):
    _phases = np.zeros(n)
    return _phases


@dispatch(int, float)
def create_initial_phases(n, phase):
    _phases = []
    for _i in range(n):
        _phases.append(phase)  # create phi of phase matrix
    del _i
    return _phases


def complex_to_real(z):      # complex vector of length n -> real of length 2n
    return np.concatenate((np.real(z), np.imag(z)))
def real_to_complex(z):      # real vector of length 2n -> complex of length n
    return z[:len(z)//2] + 1j * z[len(z)//2:]


def the_sum(arr):
    return np.sum(arr[0:]**2)

In [ ]:
N = 8
Theta = cmath.pi/4
input_vector = [1, 0, 0, 0, 0, 0, 0, 0]
output_vector = [0, 0, 0, 0, 0, 0, 0, 1]
input_vector = np.array([input_vector]).T #column
output_vector = np.array([output_vector]).T #column

In [ ]:
def fun(_phis, n, theta, _input_vector, _output_vector):
    initial_phase = create_initial_phases(n)
    initial_phase_matrix = create_phase_matrix(n, initial_phase)
    _V_matrices_product = np.eye(n, dtype=complex)
    for _i in range(2 * n):  # i is a number of a layer
        _j = _i % 2  # j - even or odd layer
        _V = np.eye(n, dtype=complex)
        while _j < n - 1:  # j became channel number
            _T = np.eye(n, dtype=complex)
            _T[_j][_j] = cmath.exp(1j * _phis[_i]) * cmath.cos(theta)
            _T[_j][_j + 1] = cmath.sin(theta)
            _T[_j + 1][_j] = -cmath.exp(1j * _phis[_i]) * cmath.sin(theta)
            _T[_j + 1][_j + 1] = cmath.cos(theta)
            _V = np.dot(_V, _T)
            _j += 2
        _V_matrices_product = np.dot(_V, _V_matrices_product)
    del _i, _j, _T, _V
    final_matrix = initial_phase_matrix@_V_matrices_product #U-matrix
    output_vector_theory = final_matrix@_input_vector
    result1 = output_vector_theory - _output_vector
    result2 = complex_to_real(result1) # n complex equations -> 2n real equations
    result3 = the_sum(result2) # f1 = 0, f2 = 0, ..., fn = 0 -> f1^2 + f2^2 + ... fn^2 = 0
    return result3

x0 = np.array([1.3, cmath.pi, 0.8, 0, 1.3, cmath.pi, 0.8, 0, 1.3, cmath.pi, 0.8, 0, 1.3, cmath.pi, 0.8, 0])
res = minimize(fun, x0, args=(N, Theta, input_vector, output_vector), method='BFGS')
print(np.round(res.x, 4))